# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1]:
from collections import defaultdict
from itertools import product
from bs4 import BeautifulSoup
from numpy.random import choice
from math import log2

import re
import random
import requests
import itertools
import numpy as np
import pandas as pd

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2]:

def dna_to_rna(s):
    converter = dict(A='A', T='U', C='C', G='G')
    return "".join(converter.get(c) for c in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

fill in 

### Discussion

fill in 

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [3]:

def read_data():
    html_content = requests.get("https://raw.githubusercontent.com/csmastersUH/data_analysis_with_python_2020/master/Codon%20usage%20table.html").text
    d = BeautifulSoup(html_content, "lxml").find("pre").text
    replace_list = ['(', ')', '\n']
    for r in replace_list:
        d = d.replace(r, ' ')
    L = np.array(d.split(' '))
    L = L[L != '']
    return np.reshape(L, (-1, 5))

def get_dict():
    L = read_data()
    return dict(zip(L[:, 0], L[:, 1]))
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

fill in 

### Discussion

fill in 

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [4]:

def get_dict_list():
    L = read_data() # Uses [14]: read_data()
    amino_dict = defaultdict(list)
    for i, amino in enumerate(L[:, 1]):
        amino_dict[amino].append(L[i, 0])
    return dict(amino_dict)
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

fill in 

### Discussion

fill in 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [5]:

def rna_to_prot(s):
  d = get_dict()
  res = ''
  for i in range(0, len(s), 3): # 3 length of codon i.e. 'UAC'
    res += d[s[i:i+3]]
  return res

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

fill in 

### Discussion

fill in 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [6]:

def get_probabability_dict():
    df = pd.DataFrame(read_data(), columns=['Codon', 'Amino', 'Fraction', 'Freq1000', 'Number'])
    df['Freq1000'] = df['Freq1000'].astype(float)
    max_freq = df.groupby('Amino').sum()
    df = df.merge(max_freq, right_on='Amino', left_on='Amino', how='outer')
    df['Fraction'] = (df['Freq1000_x'] / df['Freq1000_y'])
    return dict(zip(df['Codon'], df['Fraction']))
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.433393	AAC: 0.529086	AAG: 0.566607	AAU: 0.470914	ACA: 0.283835	ACC: 0.355263
ACG: 0.114662	ACU: 0.246241	AGA: 0.215168	AGC: 0.240444	AGG: 0.211640	AGU: 0.149199
AUA: 0.169300	AUC: 0.469526	AUG: 1.000000	AUU: 0.361174	CAA: 0.264516	CAC: 0.580769
CAG: 0.735484	CAU: 0.419231	CCA: 0.276596	CCC: 0.324059	CCG: 0.112930	CCU: 0.286416
CGA: 0.109347	CGC: 0.183422	CGG: 0.201058	CGU: 0.079365	CUA: 0.071856	CUC: 0.195609
CUG: 0.395210	CUU: 0.131737	GAA: 0.422741	GAC: 0.535181	GAG: 0.577259	GAU: 0.464819
GCA: 0.227994	GCC: 0.399711	GCG: 0.106782	GCU: 0.265512	GGA: 0.250000	GGC: 0.336364
GGG: 0.250000	GGU: 0.163636	GUA: 0.116969	GUC: 0.238880	GUG: 0.462932	GUU: 0.181219
UAA: 0.294118	UAC: 0.556364	UAG: 0.235294	UAU: 0.443636	UCA: 0.150432	UCC: 0.218249
UCG: 0.054254	UCU: 0.187423	UGA: 0.470588	UGC: 0.543103	UGG: 1.000000	UGU: 0.456897
UUA: 0.076846	UUC: 0.535620	UUG: 0.128743	UUU: 0.464380


### Idea of solution

def get_probabability_dict():


L = read_data()

    
return dict(zip(L\[:, 0], L\[:, 2])) # Probabilities are allready given in data table

### Discussion

fill in 

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [7]:
class ProteinToMaxRNA:

    def __init__(self):
        self.a_to_c = get_dict_list() # aminoacids to codon dict
        self.prob = get_probabability_dict()
      
    def convert(self, s):
        res = ''
        for amino in s:
            temp = dict()
            for codon in self.a_to_c[amino]:
                temp[codon] = self.prob[codon]
            res += max(temp, key=temp.get)
        return res


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

fill in 

### Discussion

fill in 

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [8]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    r = random.uniform(0, 1)
    sum = 0
    for e in list(dist.keys()):
      sum += dist[e]
      if r < sum:
        return e
    
    return 'T'

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, C, C, T, T, A, T, A, T, T, T, T, A, T, C, G, G, G, T, T, C, T, T, T, T, G, T, C, C


### Idea of solution

fill in 

### Discussion

fill in 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [9]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        self.a_to_c = get_dict_list() # aminoacids to codon dict
        self.prob = get_probabability_dict()

    def convert(self, s):
        res = ''
        for amino in s:
            temp = dict()
            for codon in self.a_to_c[amino]:
                temp[codon] = self.prob[codon]
            res += random_event(temp)
        return res
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUCACUCCCAUUCAAAAUAGAGCC


### Idea of solution

fill in 

### Discussion

fill in 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [3]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    for i, _ in enumerate(s):
        temp = s[i:i+k]
        if len(temp) != k: break
        yield { str(n): temp.count(n) for n in 'CGAT' } # where n stands for nucleotide
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'C': 3, 'G': 0, 'A': 0, 'T': 1}
{'C': 3, 'G': 1, 'A': 0, 'T': 0}
{'C': 2, 'G': 1, 'A': 1, 'T': 0}
{'C': 2, 'G': 1, 'A': 1, 'T': 0}
{'C': 1, 'G': 2, 'A': 1, 'T': 0}
{'C': 1, 'G': 2, 'A': 1, 'T': 0}
{'C': 2, 'G': 2, 'A': 0, 'T': 0}
{'C': 2, 'G': 2, 'A': 0, 'T': 0}
{'C': 2, 'G': 1, 'A': 0, 'T': 1}
{'C': 2, 'G': 0, 'A': 0, 'T': 2}
{'C': 1, 'G': 1, 'A': 0, 'T': 2}
{'C': 1, 'G': 1, 'A': 0, 'T': 2}
{'C': 2, 'G': 1, 'A': 0, 'T': 1}


### Idea of solution

fill in 

### Discussion

fill in 

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [11]:
def context_list(s, k):
    res = defaultdict(str)
    for i in range(len(s) - k):
        res[s[i:i+k]] += s[i+k]
    return dict(res)
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

fill in

### Discussion

fill in

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [12]:
def context_probabilities(s, k):
    prob_list = context_list(s, k)
    for k in prob_list:
        prob_list[k] = { str(n): prob_list[k].count(n) / len(prob_list[k]) for n in 'ACGT' }
    return prob_list
    
if __name__ == '__main__':
    s = 'ATGATATCATCGACGATGTAG'
    K = [0, 2]
    for k in K:
        print(context_probabilities(s, k))

{'': {'A': 0.3333333333333333, 'C': 0.14285714285714285, 'G': 0.23809523809523808, 'T': 0.2857142857142857}}
{'AT': {'A': 0.2, 'C': 0.4, 'G': 0.4, 'T': 0.0}, 'TG': {'A': 0.5, 'C': 0.0, 'G': 0.0, 'T': 0.5}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.6666666666666666}, 'TA': {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.5, 'C': 0.0, 'G': 0.5, 'T': 0.0}, 'CA': {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0}, 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0}, 'GT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}}


### Idea of solution

fill in

### Discussion

fill in

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [13]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        random.seed(seed)

        if n == 0: return ''
        if n < self.k: return "".join([random_event(self.zeroth) for _ in range(n)])
        
        seq = "".join([random_event(self.zeroth) for _ in range(self.k)])
        for _ in range(n - self.k):
            try:
                seq += random_event(self.kth[ seq[-self.k:] ])
            except KeyError as _:
                print('Key not found! ', seq[-self.k:])# Exercise 13 will.. 
                seq += random_event(self.zeroth) # handle this error
        return seq

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 1
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

Key not found!  AG
Key not found!  GG
Key not found!  GC
Key not found!  CT
Key not found!  TT
Key not found!  TT
AGGCTTTGAC


### Idea of solution

fill in

### Discussion

fill in 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [14]:
def context_pseudo_probabilities(s, k):
    comb = itertools.combinations_with_replacement(['A', 'C', 'G', 'T'], k)

    prob_list = context_list(s, k)
    for k in prob_list:
        prob_list[k] = { str(n): (prob_list[k].count(n)+1) / (len(prob_list[k])+4) for n in 'ACGT' }
    
    res = defaultdict(dict, prob_list)
    for i in list(comb): 
        for perm in itertools.permutations("".join(i)):
            if "".join(perm) not in res:
                res["".join(perm)] = {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
    return dict(res)
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.3333333333333333}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
GC: {'A': 0.25, 'C': 0.25, 'T': 0.25, 

### Idea of solution

fill in 

### Discussion

fill in 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [15]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = np.prod([self.zeroth[n] for n in s[0:self.k]])
        for i in range(len(s) - self.k):
            prob *= self.kth[ s[i:i+self.k] ][ s[i+self.k] ] # kth[ kmer ][ k ] => odd
        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

fill in

### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [16]:
class MarkovLog(object):
    def __init__(self, k, zeroth, kth):
            self.k = k
            self.zeroth = {k: log2(v) for k, v in zeroth.items()}
            self.kth = { k: { kk: log2(vv) for kk, vv in v.items() } for k, v in kth.items() }
        
    def log_probability(self, s):
        prob = sum([self.zeroth[n] for n in s[0:self.k]])
        for i in range(len(s) - self.k):
            prob += self.kth[ s[i:i+self.k] ][ s[i+self.k] ] # kth[ kmer ][ k ] => odd
        return prob
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [17]:
def better_context_probabilities(s, k):
  res = defaultdict(dict)
  if k == 0: return dict({"": { str(n): (len(re.findall(rf'{n}', s))+1) / (len(re.findall(rf'[A-Z]', s))+4) for n in 'ACGT' }})
  
  for i in itertools.combinations_with_replacement(['A', 'C', 'G', 'T'], k):
    for j in itertools.permutations("".join(i)):
      kmer = "".join(j)
      if kmer not in s[ :-1]:
        res[kmer] = {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
        continue
      # Below: Executed when kmer is in string
      res[kmer] = { str(n): (len(re.findall(rf'{kmer}{n}', s))+1) / (len(re.findall(rf'{kmer}', s))+4) for n in 'ACGT' }
  return dict(res)

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
AG: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
CC: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
GC: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
GG: {'A': 0.25, 'C': 0.25, 'T': 0.25, 'G': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [18]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.kth = better_context_probabilities(s, k)
        self.zeroth = better_context_probabilities(s, 0)[""]

    def generate(self, n, seed=None):
        random.seed(seed)

        if n == 0: return ''
        if n < self.k or 0 == self.k: return ''.join([random_event(self.zeroth) for _ in range(n)])
        seq = ''.join([ random_event(self.zeroth) for _ in range(self.k) ]) # First kmer

        for _ in range(n - self.k):
            seq += random_event(self.kth[ seq[-self.k:] ])
        return seq
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))


CATAGCAGAG


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [19]:
def kmer_index(s, k):
  res = defaultdict(list)
  for i in range(len(s)-k+1):
    res[s[i:i+k]].append(i) 
  return dict(res)

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print(len(s))
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

21
Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

fill in

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [20]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    freq = defaultdict(int)
    L = [ rna[i*3:i*3+3] for i in range(len(rna)//3) ]
    l = len(L)
    for item in L: 
        freq[item] += 1
    return {"".join(codon): (freq["".join(codon)] / l) for codon in product("ACGU", repeat=3)}

def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    sum = 0
    for k, v in p.items():
        div = 0
        try:
            div = v/q[k]
        except ZeroDivisionError as e:
            raise e # required to pass tests

        try:
            sum += v * log2(div)
        except ValueError as e:
            pass # required to pass tests
    return sum
    

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))  
    rna = ProteinToRandomRNA().convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    if protein != rna_to_prot(rna):
        print('RNA is not backward convertable with protein.')
        raise Exception()
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna) # placeholder call

    # Calculate codon probabilities based on the codon usage table
    df = pd.DataFrame(read_data(), columns=['Codon', 'Amino', 'Fraction', 'Freq1000', 'Number'])
    cp_orig = dict(zip(df['Codon'], df['Freq1000'].astype(float)/1000)) # freq per 1 codon

    # Create a completely random RNA sequence and get the codon probabilities
    rand_rna = "".join(choice(list('ACGU'), n*3))
    cp_uniform = codon_probabilities(rand_rna) # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = 0.21155841992714577
d(predicted || original) = 0.29315606468770994

d(original || uniform) = 0.206646828353321
d(uniform || original) = 0.2790378512999627

d(predicted || uniform) = 0.2671177636639163
d(uniform || predicted) = 0.2920527149948751


### Idea of solution

Better solution is put both errors under the same try except. But for some reason tests won't pass.

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [21]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    """
    https://stackoverflow.com/a/58334399
    """
    w, v = np.linalg.eig(transition.T)
    v = v[:,np.isclose(w, 1)]
    for i in range(np.size(v, 1)):
        v[:, i] = v[:,i] / v[:,i].sum()
        v[:, i] = v[:, i].real
    return v.T
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.333, +0.000, +0.667, +0.000
-0.000, +0.250, -0.000, +0.750


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [22]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    prefixes = [1, 10, 100, 1000, 10000]
    zeroth = { c: initial[i] for i, c in enumerate("ACGT")}
    kth = defaultdict(dict)
    for i, c in enumerate("ACGT"):
        kth[c] = { cc: transition[i][ii] for ii, cc in enumerate('ACGT')}
            
    kth= dict(kth)
    s = MarkovChain(k=1, zeroth=zeroth, kth=kth).generate(n=10000)
    
    res = []
    for pre in prefixes:
        d = { c: (s[:pre].count(c)+1) / (len(s[:pre])+4) for c in "ACGT" }
        res = np.append(res, kullback_leibler( { c: initial[i] for i, c in enumerate("ACGT")}, d ))

    return zip(prefixes, res)

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.76064997
KL divergence of stationary distribution prefix of length    10 is 0.55279200
KL divergence of stationary distribution prefix of length   100 is 0.02810603
KL divergence of stationary distribution prefix of length  1000 is 0.00348937
KL divergence of stationary distribution prefix of length 10000 is 0.00028846


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [425]:
def main(transition, equilibrium_distribution):
    pass


if __name__ == "__main__":
    pass

### Idea of solution

fill in

### Discussion
fill in